In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/labs/amit/noamsh/repos/sc_clustering')

In [ ]:
import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np
from sklearn import metrics

import scvi

from pathlib import Path

In [ ]:
import config

In [4]:
result_path = Path(config.RESULTS_DIR, "simple_clustering_2021_06_17__16_16_30", "loaded_data.h5ad")
adata = ad.read(result_path)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/labs/amit/noamsh/clustering_results/simple_clustering_2021_06_17__16_16_30/loaded_data.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [5]:
sc.pp.filter_genes(adata, min_counts=3)

In [6]:
adata.layers["counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata # freeze the state in `.raw`

In [7]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=1200,
    subset=True,
    layer="counts",
    flavor="seurat_v3",
)

In [8]:
scvi.__version__

'0.11.0'

In [9]:
scvi.data.setup_anndata(adata)

INFO     No batch_key inputted, assuming all cells are same batch                            
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 52608 cells, 1200 vars, 1 batches,
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              
INFO     Please do not further modify adata until model is trained.                          


This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`


In [10]:
model = scvi.model.SCVI(adata)
model

Make sure the registered X field in anndata contains unnormalized count data.


SCVI Model with the following params: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: zinb, latent_distribution: normal
Training status: Not Trained

To print summary of associated AnnData, use: scvi.data.view_anndata_setup(model.adata)

In [11]:
model.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Epoch 1/152:   0%|          | 0/152 [00:00<?, ?it/s]

The value argument must be within the support of the distribution


Epoch 152/152: 100%|██████████| 152/152 [27:47<00:00, 10.97s/it, loss=77.3, v_num=1]


In [ ]:
mode